In [1]:
import utils
import visualization
import numpy as np
import os
from IPython.display import display, Video
from dec_tswap.example_agent import RandomAgent, SmartRandomAgent, ShortestPathAgent
from dec_tswap.path_table import PathTable
from dec_tswap.base_tswap_agent import BaseTSWAPAgent, BaseTSWAPParams
from manavlib.common.params import ExperimentParams, BaseDiscreteAgentParams
import manavlib.io.xml_io as new_io

In [ ]:
# List of supported algorithm names for the experiment.
ALGORITHMS_NAMES = [
    "random",  # Uses random actions
    "smart_random",  # Uses random safe actions
    "shortest_path",  # Follows the shortest path to closes goal
    "base_tswap",  # Semi-decentralized TSWAP, requires consistent initial assignment
    "base_tswap_centr",  # Centralized TSWAP, requires consistent initial assignment
    "naive_dec_tswap",  # Naive fully decentralized TSWAP
    "dec_tswap",  # Fully decentralized TP-SWAP
]

ALGORITHM_NAME = ALGORITHMS_NAMES[
    6
]  # Select the algorithm to be used (dec_tswap in this case).
TASK_SUFFIX = "_task.xml"  # Set task file naming convention with a suffix (_task.xml).
TASK_DIR = "../tasks/example"  # Define the directory path where task files are located.
MAP_PATH = os.path.join(
    TASK_DIR, "map.xml"
)  # Specify the paths for the map and configuration files.
CONFIG_PATH = os.path.join(TASK_DIR, f"{ALGORITHM_NAME}_config.xml")
TASK_ID = 0  # Select TASK_ID for the experiment, indicating the specific task to load.

In [3]:
agents_num = 10  # Set the number of agents for the experiment.
exp_params, alg_params = new_io.read_xml_config(
    CONFIG_PATH
)  # Read experiment and algorithm parameters from the XML configuration file.
h, w, cs, grid_map = new_io.read_xml_map(
    MAP_PATH
)  # Load map data from the XML map file.
exp_params.max_steps = 1000  # Set the maximum number of steps for the experiment.

In [4]:
task_file = f"{TASK_ID}{TASK_SUFFIX}"
task_path = os.path.join(TASK_DIR, task_file)

# Read the start positions, goal positions, and agent-specific parameters from the XML task file.
default_params, starts, goals, ag_params = new_io.read_xml_agents(task_path)

# Precompute paths to a subset of goal positions (for the given number of agents).
path_table = PathTable(grid_map, goals[:agents_num])

In [5]:
print(utils.Summary.header())

summary, steps_log, neighbors_log = utils.run_experiment(
    starts,
    goals,
    grid_map,
    cs,
    agents_num,
    utils.get_agent_type(type(alg_params)),
    ag_params,
    alg_params,
    exp_params,
    path_table,
    True,
)

print(summary)

success  collision  collision_obst   makespan   flowtime    runtime     mean_groups     mean_groups_size     number
      1          0               0        143        535      0.111           8.210                 1.24         10


In [6]:
output_dir = "img"
output_filename = "animated_trajectories"
output_ext = "mp4"
output_path = os.path.join(output_dir, f"{output_filename}.{output_ext}")
visualization.draw(
    grid_map, goals, steps_log, neighbors_log, 20, 20, output_path
)
display(Video(filename=output_path))